In [2]:
import csv
import PIL
from PIL import Image
from tensorflow import keras
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageTk 
from tkinter import *
from tkinter import filedialog

csvfiles = {
    1: 'UFPR04.csv',
    2: 'UFPR05.csv',
    3: 'PUCPR.csv'
}
root = Tk()
root.title('Smart Parking')
root.geometry("1070x740")

def browseFile():
    global myIimage
    
    root.filename = filedialog.askopenfilename(initialdir='/', title='Select a file',
                                               filetypes=(('JPG files', '*.jpg'),
                                                          ('All files', '*.*')))
    img = PIL.Image.open(root.filename)
    UFPR04parking(img)

def UFPR04parking(im):
    global model
    spots_csv = open(csvfiles[v.get()], mode='r')
    csv_reader = csv.DictReader(spots_csv)

    occupied = []
    available = []
    model = tf.keras.models.load_model('XXX.h5')
    draw = ImageDraw.Draw(im)
    for row in csv_reader:
        spot_img = im.crop((int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])))
        newsize = (40, 32)
        spot_img = spot_img.resize(newsize)
        result = makePrediction(spot_img,model)
        if result == 1:
            occupied.append(row["id"])
            draw.rectangle([int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])], fill=None, outline='red', width=1)
        else:
            available.append(row["id"])
            draw.rectangle([int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])], fill=None, outline='green', width=1)
    mytext = f"{len(occupied)} occupied, {len(available)} available \n These spots are available {available}"
    label4["text"] = mytext
    im = im.resize((1000, 563), PIL.Image.ANTIALIAS)
    myImage = ImageTk.PhotoImage(im)
    imageLabel.configure(image=myImage)
    imageLabel.photo = myImage

def makePrediction(spot_img, model):
    img_array = keras.preprocessing.image.img_to_array(spot_img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    result = model.predict(img_array, steps=1)
    result = np.rint(result)
    if (result):
        return 1
    else:
        return 0

def radioSelected():
    myButton['state'] = 'normal'

v = IntVar()
v.set("L") # initialize

# Setup the GUI widgets
# Labels
label1 = Label(root, text="Step1: Select Parking", font='Helvetica 14 bold')
label2 = Label(root, text="Step2: Upload an Image", font='Helvetica 14 bold')
label3 = Label(root, text="Result:", font='Helvetica 14 bold')
label4 = Label(root, text="", wraplength=350)
label1.grid(row=0, column=0)
label2.grid(row=0, column=1)
label3.grid(row=0, column=2)
label4.grid(row=1, column=2, rowspan=3)
# Image
imageLabel = Label(root, text="")
imageLabel.grid(row=5, column=0, columnspan=3, padx=(35, 35))
# Radio Buttons
R1 = Radiobutton(root, text="Parking 1", variable=v, value=1, command = lambda : radioSelected())
R2 = Radiobutton(root, text="Parking 2", variable=v, value=2, command = lambda : radioSelected())
R3 = Radiobutton(root, text="Parking 3", variable=v, value=3, command = lambda : radioSelected())
R1.grid(row=1, column=0)
R2.grid(row=2, column=0)
R3.grid(row=3, column=0)
# Upload Button
myButton = Button(root, text='Open File', command=browseFile, state=DISABLED)
myButton.grid(row=1, column=1)
# Horizontal Line
f = Frame(root,height=2,width=1070,bg="black")
f.grid(row=4, column=0, columnspan=3)


root.mainloop()